In [ ]:
import sys
print(sys.executable)


c:\Users\sameer\anaconda3\envs\torch312\python.exe


In [1]:
import torch
print(torch.__version__)  # coCheck PyTorch version
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should print GPU name



2.5.1+cu121
True
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extract visible text from paragraphs and headings
        text = " ".join([p.get_text() for p in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"])])
        return text
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return None

# Example: Scrape multiple URLs
urls = [
    "https://en.wikipedia.org/wiki/Hampi",
    "https://whc.unesco.org/en/list/241/#",
    "https://openthemagazine.com/essay/the-untold-history-of-hampi/",
    #"https://www.opindia.com/2018/02/the-ruins-of-hampi-a-song-of-hope-and-a-story-of-treachery-then-and-now/",
    "https://en.wikipedia.org/wiki/Virupaksha_Temple,_Hampi",
    "https://www.thrillophilia.com/questions/why-is-hampi-called-the-city-of-ruins",
    "https://unacademy.com/content/bank-exam/study-material/general-awareness/group-of-monuments-at-hampi/#:~:text=Hampi%2C%20also%20known%20as%20the,like%20the%20lotus%20and%20corbels."
    
]
documents = {url: extract_text_from_url(url) for url in urls}

# Remove empty results
documents = {url: text for url, text in documents.items() if text}



In [9]:
!pip uninstall -y huggingface-hub
!pip install huggingface_hub==0.10.1
!pip install --upgrade sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np
import torch

# Load a pre-trained embedding model and move it to GPU
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("Model loaded successfully!")

# Example embeddings (ensure `documents` is defined in your code)
embeddings = np.array([model.encode(text, device=model.device) for text in documents.values()])
print("Embeddings shape:", embeddings.shape)

Found existing installation: huggingface-hub 0.29.3
Uninstalling huggingface-hub-0.29.3:
  Successfully uninstalled huggingface-hub-0.29.3
  Using cached huggingface_hub-0.10.1-py3-none-any.whl.metadata (6.1 kB)
Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.10.1 which is incompatible.
tokenizers 0.21.1 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 0.10.1 which is incompatible.
transformers 4.49.0 requires huggingface-hub<1.0,>=0.26.0, but you have huggingface-hub 0.10.1 which is incompatible.


  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.29.3-py3-none-any.whl (468 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
Model loaded successfully!
Embeddings shape: (6, 384)


In [10]:
import chromadb

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create a collection
collection = chroma_client.get_or_create_collection(name="text_embeddings")

# Add data to the database
for i, (url, text) in enumerate(documents.items()):
    collection.add(
        ids=[str(i)],  
        documents=[text], 
        embeddings=[embeddings[i].tolist()]
    )




print("Stored in ChromaDB!")


Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 1


Stored in ChromaDB!


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

# Use Hugging Face embeddings instead of OpenAI
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load stored vectors
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

# Use Groq API key
groq_api_key = "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"

# Define Retrieval-Augmented Generation (RAG) pipeline using Groq
qa = RetrievalQA.from_chain_type(
    llm=ChatGroq(model="llama3-8b-8192", groq_api_key=groq_api_key),  # Change model as needed
    retriever=vectorstore.as_retriever()
)

def is_relevant_to_hampi(prompt):
    keywords = ["Hampi", "monument", "Vijayanagara", "temple"]
    return any(keyword.lower() in prompt.lower() for keyword in keywords)

# # Query the database
prompt = input("Enter the prompt: ")

if is_relevant_to_hampi(prompt):
    result = qa.run(prompt)
    print(result)
else:
    print("Your query is not relevant to Hampi cultural sites.")

The Stone Chariot in Hampi is a magnificent ancient structure located in the ancient city of Vijayanagara, which is now a UNESCO World Heritage site in Karnataka, India. The chariot is a unique monument that stands out for its intricate carvings and sculptures.

The Stone Chariot is believed to have been built during the reign of King Krishnadevaraya (1509-1529 CE) of the Vijayanagara Empire. It is a freestanding structure, about 7 meters (23 feet) tall, with four wheels and a platform. The chariot is made of a single block of granite, weighing around 80 tons.

The chariot is decorated with intricate carvings and sculptures, showcasing the artistic skills of the ancient sculptors. The sculptures depict various mythological scenes, including gods and goddesses, animals, and mythological creatures. Some of the notable sculptures include:

* The figure of Lord Shiva, standing on the platform of the chariot, surrounded by his consorts, Parvati and Uma.
* The depiction of the god of war, Lo